In [78]:
%load_ext autoreload
%autoreload 1
import sys; sys.path.append("..")
%aimport lib.util
from lib.util import reformat_tract, add_race_eth, add_purpose_type, add_income_group 
#add_all_category_columns
import pandas as pd
pd.options.display.max_columns = 50
pd.options.display.max_rows = 1500
import numpy as np
from ambry import library
l = library()
p = l.get('ffiec.gov-hmda-2010e-lar-2012-ca').partition
df = p.select("SELECT * FROM lar WHERE county_code = 73").pandas

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [79]:
add_race_eth(df) # Adds race_eth column
add_purpose_type(df) # Adds purpose_type column Combines loan_purpose and loan_type in the same way as the summary report
add_income_group(df) # Adds income_group column
#add_all_category_columns(df) # Add '_c' categorical columns

In [80]:
def summaryTable(newdf, actiontype):
    totalcount = newdf.loan_amount.count()
    totalamount = newdf.loan_amount.sum()
    ### By Loan Purpose and Type
    loanpt = newdf.groupby(['purpose_type'])
    loanpt = loanpt['loan_amount'].agg(
                            {'median':np.median,
                             'count': np.size,
                             'amount': np.sum})
    loanpt['%count'] = np.round(100*loanpt['count']/totalcount,1)
    loanpt['%amount'] = np.round(100*loanpt['amount']/totalamount,1)
    loanpt=loanpt[['median','count','%count','amount','%amount']]
    ### Subtotals
    
    ### By Applicant Race/Ethnicity
    race_eth  = newdf.groupby(['race_eth'])
    race_eth  = race_eth ['loan_amount'].agg(
                            {'median':np.median,
                             'count': np.size,
                             'amount': np.sum})
    race_eth ['%count'] = np.round(100*race_eth['count']/totalcount,1)
    race_eth ['%amount'] = np.round(100*race_eth['amount']/totalamount,1)
    race_eth = race_eth [['median','count','%count','amount','%amount']]
    ###By Applicant Income
    income  = newdf.groupby(['income_group'])
    income   = income['loan_amount'].agg(
                            {'median':np.median,
                             'count': np.size,
                             'amount': np.sum})
    income ['%count'] = np.round(100*income['count']/totalcount,1)
    income ['%amount'] = np.round(100*income['amount']/totalamount,1)
    income = income [['median','count','%count','amount','%amount']]
    ###By Applicant Sex
    sex  = newdf.groupby(['applicant_sex'])
    sex   = sex['loan_amount'].agg(
                            {'median':np.median,
                             'count': np.size,
                             'amount': np.sum})
    sex ['%count'] = np.round(100*sex['count']/totalcount,1)
    sex ['%amount'] = np.round(100*sex['amount']/totalamount,1)
    sex = sex [['median','count','%count','amount','%amount']]
    ###Owner-Occupied
    occ = newdf.groupby(['occupancy'])
    occ   = occ['loan_amount'].agg(
                            {'median':np.median,
                             'count': np.size,
                             'amount': np.sum})
    occ ['%count'] = np.round(100*occ['count']/totalcount,1)
    occ ['%amount'] = np.round(100*occ['amount']/totalamount,1)
    occ = occ [['median','count','%count','amount','%amount']]
  
    ### Smashing Tables Together
    frames = [loanpt, race_eth,income,sex,occ]
    table = pd.concat(frames, keys=['Loan Purpose and Type', 
                                   'Applicant Race/Eth', 
                                   'Applicant Income',
                                   'Applicant Sex',
                                   'Occupancy'])
    pieces = [table.ix[:, ["median","count","%count","amount","%amount",]]]
    table = pd.concat(pieces, axis=1, keys=[actiontype])
    return table;

# All Lenders Combined

In [81]:
#Total -- All types
newdf=df
total=summaryTable(newdf,"Total")

#1 -- Loan originated
newdf = df[df.action_type==1]
action1 = summaryTable(newdf,"Loan originated")

#2 -- Application approved but not accepted
newdf = df[df.action_type==2]
action2 = summaryTable(newdf,"Application approved but not accepted")

#3 -- Application denied by financial institution
newdf = df[df.action_type==3]
action3 = summaryTable(newdf,"Application denied by financial institution")

#4 -- Application withdrawn by applicant
newdf = df[df.action_type==4]
action4 = summaryTable(newdf,"Application denied by financial institution")

#5 -- File closed for incompleteness
newdf = df[df.action_type==5]
action5 = summaryTable(newdf,"File closed for incompleteness")

#6 -- Loan purchased by the institution
newdf = df[df.action_type==6]
action6 = summaryTable(newdf,"Loan purchased by the institution")

#Merging All Tables
all_lenders = pd.concat([total,action1,action2,action3,action4,action5,action6], axis=1)
all_lenders

Total                            \
                                             median   count  %count    amount   
Loan Purpose and Type improvement             161.0    4781     2.0   1038687   
                      purchase-conventional   304.0   32857    13.5  12419787   
                      purchase-govt           320.0   22240     9.2   7551607   
                      refinance               297.0  182905    75.3  61055963   
Applicant Race/Eth    aian                    280.0     767     0.3    231319   
                      as                      308.0   23920     9.9   7808190   
                      b                       270.0    4053     1.7   1139822   
                      h                       251.0   24012     9.9   6385159   
                      pi                      293.0    1908     0.8    570463   
                      w                       305.0  142000    58.5  48384565   
Applicant Income      high                    787.5    1198     0.5   1344670   
                      low                     292.0  207208    85.3  64534660   
                      mid                     646.0    1557     0.6   1344339   
                      mod                     546.0    4278     1.8   2947579   
Applicant Sex         1                       312.0  159485    65.7  54930806   
                      2                       258.0   50394    20.8  14172171   
                      3                       295.0   17527     7.2   5939329   
                      4                       300.0   15377     6.3   7023738   
Occupancy             1                       307.0  208353    85.8  70232806   
                      2                       240.0   33252    13.7   9353423   
                      3                       825.0    1178     0.5   2479815   

                                                      Loan originated          \
                                             %amount           median   count   
Loan Purpose and Type improvement                1.3            206.0    2315   
                      purchase-conventional     15.1            313.0   18452   
                      purchase-govt              9.2            325.0   10545   
                      refinance                 74.4            296.0  103541   
Applicant Race/Eth    aian                       0.3            280.0     417   
                      as                         9.5            313.5   14274   
                      b                          1.4            273.0    2219   
                      h                          7.8            252.0   12799   
                      pi                         0.7            297.0    1027   
                      w                         59.0            303.0   86821   
Applicant Income      high                       1.6            921.5     698   
                      low                       78.6            292.0  117924   
                      mid                        1.6            698.0     916   
                      mod                        3.6            546.0    2564   
Applicant Sex         1                         66.9            312.0   95760   
                      2                         17.3            258.0   29277   
                      3                          7.2            301.0    8844   
                      4                          8.6            864.0     972   
Occupancy             1                         85.6            308.0  115835   
                      2                         11.4            242.0   18108   
                      3                          3.0            906.0     910   

                                                                        \
                                             %count    amount  %amount   
Loan Purpose and Type improvement               1.7    624517      1.3   
                      purchase-conventional    13.7   7434158     16.0   
                      

In [82]:
all_lenders.to_csv('allLenders.csv')